In [46]:
from astropy.io import fits
import sep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse 
%matplotlib inline

In [47]:
#Load the fits file
def load_fits(fileName):
    hduList = fits.open(fileName) #Open the file
    hduList = fits.open(fileName) #Open the file
    #print(hduList.info()) 
    data = np.array(hduList[0].data)
    data = data.byteswap(inplace=True).newbyteorder()
    hduList.close() #Close the file
    #Summary Statistics
    print('Summary', fileName, '| Min:', np.min( data), "| Max:", np.max( data), "| Mean:", np.mean(  data), "| Stdev:", np.std( data))
    return data

#Plot the fits file
def plot_data(fileName, data):
    m, s = np.mean(data), np.std(data)
    plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
    plt.title(f"Image for {fileName}")
    plt.colorbar()

In [48]:
#Estimate + plot background noise
def estimate_background(data):
    bkg = sep.Background(data) #estimate background noise => background object
    print(f"Background mean: {bkg.globalback} | Background RMS: {bkg.globalrms}")
    bkg_image = bkg.back() # convert to 2d array, equal to np.array(bkg)
    #plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower') #plot background noise
    #plt.colorbar()
    return bkg

#Subtract background noise from data 
def subtract_background(data, bkg):
    data_sub = data - bkg # subtract the background
    m, s = np.mean(data_sub), np.std(data_sub)
    #plt.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
    #plt.colorbar()
    return data_sub

def calculate_snr(data, bkg):
    signal = np.mean(data) - bkg.globalback  # Calculate the signal as the mean of the data minus the background level
    noise = bkg.globalrms  # Estimate the noise as the background RMS
    snr = signal / noise  # Calculate the SNR
    return snr

In [49]:
#Count stars from the data 
def extract_objects(data_sub, bkg):
    thresh = 1.5 # Set the detection threshold to be 1.5 sigma above the background
    objects, segmap = sep.extract(data_sub, thresh, err=bkg.globalrms, segmentation_map=True) # detect objects with a detection threshold of 1.5 sigma, where sigma = globalrms
    
    #Filter out the objects that are too close to the edge of the image
    edge_lim = 50 # This sets how far off the image edges we start to include stars
    cond1 = (objects['xpeak'] > edge_lim) & (objects['xpeak'] < data_sub.shape[0] - edge_lim) & \
       (objects['ypeak'] > edge_lim) & (objects['ypeak'] < data_sub.shape[1] - edge_lim) # If centroids are within the edge limit, then it is a true detection 
       
    #Filter out hot pixels : Check if the median close to the target pixel is more than thresh*background RMS
    #If the median of the area around the target pixel is brighter than the background RMS, then it is a true detection
    cond2 = np.zeros(cond1.shape)
    for i in range (cond2.shape[0]):
        # Check the median close to the target pixel
        close_median = np.median(data_sub.T[objects['xpeak'][i] - 1 : objects['xpeak'][i] + 2,
                                                    objects['ypeak'][i] - 1 : objects['ypeak'][i] + 2])
        if close_median > thresh * bkg.globalrms:
            cond2[i] = 1   # If exceeds, then it is a true detection
    cond2 = cond2.astype(bool) # Convert to boolean mask
    cond_final = np.logical_and(cond1,cond2) # Combine the two conditions 
    objects = objects[cond_final] # Apply the combined mask to the objects array
    print("Stars: ", len(objects)) 
    return np.array(objects) #, segmap

#Plot the stars 
def plot_stars(file, data_sub, objects, segmap):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    # Plot the background-subtracted image with ellipses
    ax1.imshow(data_sub, cmap='gray', origin='lower')
    ax1.set_title('file: ' + file[14:23])

    # Plot the segmentation map without ellipses
    ax2.imshow(segmap, cmap='jet', origin='lower')
    ax2.set_title('Segmentation Map')

    # Plot ellipses around the detected stars in the background-subtracted image
    for obj in objects:
        ellipse = Ellipse(xy=(obj['x'], obj['y']),
                          width=6* obj['a'],
                          height=6 * obj['b'],
                          angle=obj['theta'] * 180.0 / np.pi)
        ellipse.set_facecolor('none')
        ellipse.set_edgecolor('yellow')
        ax1.add_artist(ellipse)
        #ax1.text(obj['x'], obj['y'], str(i+1), #code for label; not feasible for large number of stars
           #      color='white', fontsize=4, ha='center', va='center')

    plt.tight_layout()
    plt.show()

In [54]:
#dataFiles = ["data\\NGC_1245_b_120_calibrated_stacked.fit", "data\\NGC_1245_v_120_calibrated_stacked.fit", "data\\NGC_1245_b_900_calibrated_stacked.fit", "data\\NGC_1245_v_900_calibrated_stacked.fit"]

def getData(file):
  data = load_fits(file)
  bkg = estimate_background(data)
  data_sub = subtract_background(data, bkg)
  return data, data_sub, bkg


def getFluxes(data_sub, objects, bkg):
    flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)
    return flux, fluxerr, flag
  
  
#for file in dataFiles:
  #  data, data_sub, bkg = getData(file)
  #  objects, segmap = extract_objects(data_sub, bkg)
  #  plot_stars(file, data_sub, objects, segmap)
  #  print("========================================")

In [55]:
file = "data\\NGC_1245_b_900_calibrated_stacked.fit"

data, data_sub, bkg = getData(file)
objects = extract_objects(data_sub, bkg)
blueFlux, blueFluxerr, blueFlag = getFluxes(data_sub, objects, bkg)
    
file = "data\\NGC_1245_v_900_calibrated_stacked.fit"

data, data_sub, bkg = getData(file)
objects = extract_objects(data_sub, bkg)
greenFlux, greenFluxerr, greenFlag = getFluxes(data_sub, objects, bkg)

photom = blueFlux - greenFlux

sorted_indices = photom.argsort()[::-1]
sorted_flux = photom[sorted_indices]
sorted_x = objects['x'][sorted_indices]
sorted_y = objects['y'][sorted_indices]

# Plot the scatterplot
plt.scatter(sorted_x, sorted_y, c=sorted_flux, cmap='viridis')
plt.colorbar(label='Flux')
plt.title('Brightest Objects Scatterplot')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()


Summary data\NGC_1245_b_900_calibrated_stacked.fit | Min: 295.73376 | Max: 883135.3 | Mean: 5710.8955 | Stdev: 2547.5684
Background mean: 5643.23779296875 | Background RMS: 54.152748107910156


c:\Users\mmann\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\mmann\anaconda3\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Stars:  1404
Summary data\NGC_1245_v_900_calibrated_stacked.fit | Min: 425.97168 | Max: 970909.9 | Mean: 9407.472 | Stdev: 5766.7773
Background mean: 9274.0849609375 | Background RMS: 63.27387237548828
Stars:  1840


ValueError: operands could not be broadcast together with shapes (1404,) (1840,) 